testing jeopardy stuff


In [16]:

from bs4 import BeautifulSoup as bs
from urllib2 import urlopen

instead of analyzing each game, let's try to categorize some of the questions that appear.

we'll pull all the clues and set aside a separate category for all the finals.

In [41]:
urls = ["http://www.j-archive.com/showgame.php?game_id=" + str(i) for i in xrange(5000)]

clues = []; finals = []

for url in urls[:4900]:
    
    sock = urlopen(url)
    lines = sock.read()
    sock.close()
    
    soup = bs(str(lines))

    # clues
    for i in range(1,7):
        for j in range(1,7):
            try:
                clues.append(soup.find(id = 'clue_J_' + str(i) + '_' + str(j)).get_text())
                clues.append(soup.find(id = 'clue_DJ_' + str(i) + '_' + str(j)).get_text())
            except AttributeError:
                continue

    # final
    try:
        finals.append(soup.find(id = 'clue_FJ').get_text())
    except AttributeError:
        continue


In [2]:
import pickle

with open("j-clues.txt", 'rb') as f:
    clues = pickle.load(f)
f.close()

with open("j-finals.txt", 'rb') as f:
    finals = pickle.load(f)
f.close()

annoying directory nonsense

In [1]:
import sys
sys.path.append("/usr/local/lib/python2.7/site-packages")

In [12]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag

wordnet_tags = ['n','v']
def lemmatize(token, tag):
    if tag.lower() in ['n','v']:
        return lemmatizer.lemmatize(token, tag.lower())
    return token
lemmatizer = WordNetLemmatizer()
print "here?"
tagged = [pos_tag(word_tokenize(sentence)) for sentence in clues[100]]

here?


first step is to turn clues into a matrix.
rows = words, columns = sentences

In [3]:
# remove stop words from clues

from nltk.corpus import stopwords

cachedStopWords = stopwords.words("english")

def remove_stops(text):
    copy = []
    for sentence in text:
        copy.append(' '.join([word for word in sentence.split() if word not in cachedStopWords]))
    return copy

s_clues = remove_stops(clues)


In [6]:
# transform clues into vectors of 0/1 using vocabulary list above

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words = 'english')
dense = vectorizer.fit_transform(clues).todense()
words = vectorizer.vocabulary_


In [20]:
td_matrix = dense.T

In [21]:
td_matrix.shape

(39073, 37457)

In [22]:
from math import log
from numpy import asarray, sum

In [29]:
def TFIDF(mat):
    WordsPerDoc = sum(mat, axis=0) 
    DocsPerWord = sum(asarray(mat > 0, 'i'), axis=1) 
    rows, cols = mat.shape 
    for i in range(rows):
        for j in range(cols):
            try:
                mat[i,j] = (mat[i,j] / WordsPerDoc[j]) * log(float(cols) / DocsPerWord[i])
            except ValueError:
                mat[i,j] = 0

In [ ]:
from numpy import zeros
from scipy.linalg import svd

U, S, V = svd(td_matrix, full_matrices=False)

In [ ]:
print("hello")

In [41]:
# make numpy matrix of these vectors
 
